In [10]:
'''
Source
https://www.kaggle.com/code/feyzazkefe/lstm-twitter-offensive
'''
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, f1_score
from sklearn import metrics
from keras.layers import LSTM,  Dense, Dropout, Embedding, Bidirectional
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from copy import deepcopy
from warnings import filterwarnings
from sklearn.model_selection import KFold
filterwarnings("ignore")


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
datapath = "/content/drive/MyDrive/NLP/H1_Offensive_Language_Identification_train.csv"
trainset = pd.read_csv(datapath);



### **Added More Data**

In [ ]:
# uncomment lines of this cell to add additional rows in the dataset
# datapath = "/content/drive/MyDrive/NLP/H1_Offensive_Language_Identification_train.csv"
# trainset = pd.read_csv(datapath);
 


# df1 = pd.read_csv("/content/drive/MyDrive/NLP/test_a_labels_all.csv", names=['id','label'])
# df2 = pd.read_csv("/content/drive/MyDrive/NLP/test_a_tweets_all.tsv",  delimiter='\t')
# df_add = pd.merge(df1, df2, how='inner')

# trainset = pd.concat([df_add,trainset], ignore_index=True )
# trainset.reset_index(inplace=True)

In [13]:
#remove contractions
def decontract(text):
    text = text.replace(r"can't", "can not ")
    text = text.replace(r"n't", " not ")
    text = text.replace(r"'re", " are ")
    text = text.replace(r"'s ", " is ")
    text = text.replace(r"'d ", " would ")
    text = text.replace(r"'ll", " will ")
    text = text.replace(r"'t", " not ")
    text = text.replace(r"'ve", " have ")
    text = text.replace(r"'m", " am ")
    return text
    
    
def text_preprocessing(df):
    '''
    input : df is the dataframe(obtained by  pd.read_csv(H1_Offensive_Language_Identification_train.csv) ) 
    output: None . Tweets in df will be preprocessed internally.
    '''
    punctuations = string.punctuation
    
    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') #Remove mentions (@USER)
    df.loc[:, 'tweet'] = df.tweet.str.replace('URL', '') #Remove URLs
    df.loc[:, 'tweet'] = df.tweet.str.replace('&amp', 'and') #Replace ampersand (&) with and
    df.loc[:, 'tweet'] = df.tweet.str.replace('&lt','') #Remove &lt
    df.loc[:, 'tweet'] = df.tweet.str.replace('&gt','') #Remove &gt
    df.loc[:, 'tweet'] = df.tweet.str.replace('\d+','') #Remove numbers
    df.loc[:, 'tweet'] = df.tweet.str.replace('’',"'") #convert webpage apostrophe to ascii 39 
    df.loc[:, 'tweet'] = df.tweet.str.replace('‘',"'") #convert webpage apostrophe to ascii 39 
    df.loc[:, 'tweet'] = df.tweet.str.lower() #Lowercase
    for index_label, row_series in df.iterrows():
    
      df.at[index_label , 'tweet'] = decontract(row_series['tweet'])  
    # print(df.loc[11,'tweet'])
    #Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')
    # print(df)
    #Remove emojis
    
    df.loc[:, 'tweet'] = df.astype(str).apply(   lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii')  )
    # d = df.astype(str).apply(   lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii')  )
    # print(df.astype(str))
    #remove multiple spaces
    for index_label, row_series in df.iterrows():
    
      df.at[index_label , 'tweet'] = " ".join(row_series['tweet'].split(" ")) 
    df.loc[:, 'tweet'] = df.tweet.str.strip() #Trim leading and trailing whitespaces



In [14]:

text_preprocessing(trainset)

In [15]:
trainset.update(trainset[['tweet']].applymap('\'{}\''.format))  

In [16]:
encoder = LabelEncoder()
trainset["class_a_code"] = encoder.fit_transform(trainset["label"])
 

In [17]:
trainset

,tweet,label,id,class_a_code
0,'she should ask a few native americans what th...,OFF,0,1
1,'go home you are drunk maga trump',OFF,1,1
2,'amazon is investigating chinese employees who...,NOT,2,0
3,'someone should have taken this piece of shit ...,OFF,3,1
4,'obama wanted liberals and illegals to move in...,NOT,4,0
...,...,...,...,...
13235,'sometimes i get strong vibes from people and ...,OFF,13235,1
13236,'benidorm creamfields maga not too shab...,NOT,13236,0
13237,'and why report this garbage we do not give ...,OFF,13237,1
13238,'pussy',OFF,13238,1


In [18]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
trainset, testset = train_test_split(
  trainset,
  test_size=0.33,
  random_state=RANDOM_SEED
) 
X_train = trainset["tweet"]
y_train = trainset["class_a_code"]
X_test = testset["tweet"]
y_test = testset["class_a_code"]

In [19]:
print("Shape of X_train is {} and shape of y_train is {}".format(X_train.shape, y_train.shape))


Shape of X_train is (8870,) and shape of y_train is (8870,)


In [20]:
# TOKENIZER
max_features = 10000
embedding_dim = 128
max_len=500


tokenizer=Tokenizer(num_words=max_features,oov_token='</OOV>')
tokenizer.fit_on_texts(X_train.values)
dic=tokenizer.word_index

In [21]:
# TRAIN
X_train_seq = tokenizer.texts_to_sequences(X_train.values)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)

print("train data tensor:" ,X_train_pad.shape)

train data tensor: (8870, 500)


In [22]:
# TEST
X_test_seq = tokenizer.texts_to_sequences(X_test.values)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

print("test data tensor:" ,X_test_pad.shape)

test data tensor: (4370, 500)


In [23]:
model= Sequential()
model.add(Embedding(max_features,embedding_dim,input_length=max_len))
model.add(Bidirectional(LSTM(128, dropout=0.3)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.summary()

METRICS = [tf.keras.metrics.BinaryAccuracy(),
      tf.keras.metrics.Precision(name="precision"),
      tf.keras.metrics.Recall(name="recall")]
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=METRICS)
bestmodel = None
bestf1 = 0
for epochs in range(3):
# Fit the model using the train and test datasets.
  history = model.fit(X_train_pad,y_train,validation_data=(X_test_pad,y_test), epochs=epochs+1, initial_epoch=epochs)
  print(history)
  y_pred = model.predict(X_test_pad)
  for i in range(len(y_pred)):
    if(y_pred[i] > 0.5):
        y_pred[i] = 1
    else:
        y_pred[i] =0
  y_pred2 = []
  for i in y_pred:
    y_pred2.append(int(i[0]))
  y_test2 = list(y_test)
  print("f1score")
  f1 = f1_score(y_test2, y_pred2, average='macro')
  print(f1)
  if(f1 > bestf1):
    bestmodel = deepcopy(model)
    bestf1 = f1


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,576,193
Trainable params: 1,576,193
Non-trainable params: 0
______________________________________________

Epoch 2/2
137/137 [==============================] - 3s 20ms/step
f1score
0.683316299696383


Epoch 3/3
137/137 [==============================] - 3s 19ms/step
f1score
0.6786078205540971


In [ ]:
from tensorflow import keras
reconstructed_model = keras.models.load_model('/content/drive/MyDrive/NLP/lstm_model6')

In [ ]:
y_pred = reconstructed_model.predict(X_test_pad)
y_pred

27/27 [==============================] - 13s 485ms/step


array([[0.7277004 ],
       [0.17555314],
       [0.5187438 ],
       [0.6229943 ],
       [0.7645574 ],
       [0.13642685],
       [0.32613912],
       [0.40348765],
       [0.33557093],
       [0.02934941],
       [0.39720613],
       [0.48247346],
       [0.15563078],
       [0.06432112],
       [0.08428004],
       [0.16128981],
       [0.3519155 ],
       [0.7200641 ],
       [0.61644506],
       [0.03701522],
       [0.76693046],
       [0.23843016],
       [0.83130014],
       [0.4372502 ],
       [0.07588664],
       [0.3342559 ],
       [0.53734946],
       [0.9876625 ],
       [0.27336538],
       [0.0997723 ],
       [0.586621  ],
       [0.95257074],
       [0.4802803 ],
       [0.42166868],
       [0.15336488],
       [0.22911496],
       [0.3093203 ],
       [0.38282323],
       [0.3227204 ],
       [0.07459065],
       [0.07948104],
       [0.28700343],
       [0.9459591 ],
       [0.07762069],
       [0.21658671],
       [0.29548934],
       [0.1196842 ],
       [0.359

In [ ]:

for i in range(len(y_pred)):
    if(y_pred[i] > 0.5):
        y_pred[i] = 0
    else:
        y_pred[i] =1
y_pred2 = []
for i in y_pred:
  y_pred2.append(int(i[0]))
y_test2 = list(y_test)
f1_score(y_test2, y_pred2, average='macro')

0.4225803119512447

In [ ]:
y_pred = reconstructed_model.predict(X_test_pad)
y_pred2 = []
for i in y_pred:
  y_pred2.append(i[0])
import pickle
dbfile = open('lstm_predict.pkl', 'wb')
  
# source, destination
pickle.dump(y_pred2, dbfile)                     
dbfile.close()

27/27 [==============================] - 1s 23ms/step


In [ ]:
 from sklearn.metrics import confusion_matrix
 confusion_matrix(y_test2, y_pred2)

array([[530,  90],
       [ 90, 150]])

### **cross validation**

In [ ]:

kf = KFold(n_splits=3, random_state = 2, shuffle=True)
f1s = []
for train_index, test_index in kf.split(X_train):
  X_test = X_train[test_index]
  y_test = y_train[test_index]
  X_train2 = X_train[train_index]
  y_train2 = y_train[train_index]
   

  # TOKENIZER
  max_features = 10000
  embedding_dim = 128
  max_len=500

  tokenizer=Tokenizer(num_words=max_features,oov_token='</OOV>')
  tokenizer.fit_on_texts(X_train.values)
  dic=tokenizer.word_index
  # TRAIN
  X_train_seq = tokenizer.texts_to_sequences(X_train2.values)
  X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)

  print("train data tensor:" ,X_train_pad.shape)
  # TEST
  X_test_seq = tokenizer.texts_to_sequences(X_test.values)
  X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

  print("test data tensor:" ,X_test_pad.shape)
  model= Sequential()
  model.add(Embedding(max_features,embedding_dim,input_length=max_len))
  model.add(Bidirectional(LSTM(128, dropout=0.3)))
  model.add(Dense(128,activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(1,activation='sigmoid'))
  model.summary()

  METRICS = [tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall")]
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=METRICS)
  bestmodel = None
  bestf1 = 0
  for epochs in range(3):
  # Fit the model using the train and test datasets.
    history = model.fit(X_train_pad,y_train2,validation_data=(X_test_pad,y_test), epochs=epochs+1, initial_epoch=epochs)
    print(history)
    y_pred = model.predict(X_test_pad)
    for i in range(len(y_pred)):
      if(y_pred[i] > 0.5):
          y_pred[i] = 1
      else:
          y_pred[i] =0
    y_pred2 = []
    for i in y_pred:
      y_pred2.append(int(i[0]))
    y_test2 = list(y_test)
    print("f1score")
    f1 = f1_score(y_test2, y_pred2, average='macro')
    print(f1)
    if(f1 > bestf1):
      bestmodel = deepcopy(model)
      bestf1 = f1
  f1s.append(bestf1)


train data tensor: (8826, 500)
test data tensor: (4414, 500)
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,576,193
Trainable params: 1,576,193
Non-tra

Epoch 2/2
138/138 [==============================] - 3s 20ms/step
f1score
0.6856384175647358


Epoch 3/3
138/138 [==============================] - 3s 20ms/step
f1score
0.6868448680567016


train data tensor: (8827, 500)
test data tensor: (4413, 500)
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,576,193
Trainable params: 1,576,193
Non-tra

Epoch 2/2
138/138 [==============================] - 3s 19ms/step
f1score
0.6813930462769467


Epoch 3/3
138/138 [==============================] - 3s 19ms/step
f1score
0.6743988757573662
train data tensor: (8827, 500)
test data tensor: (4413, 500)
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 256)              263168    
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 129       
                                

Epoch 2/2
138/138 [==============================] - 3s 19ms/step
f1score
0.6735010906800581


Epoch 3/3
138/138 [==============================] - 3s 19ms/step
f1score
0.6808018933858817


In [ ]:
f1s

[0.6868448680567016, 0.6813930462769467, 0.6808018933858817]

In [ ]:
sum(f1s)/3

0.6830132692398433

In [ ]:
# bestmodel.save('/content/drive/MyDrive/NLP/lstm_model6')
from tensorflow import keras
reconstructed_model = keras.models.load_model('/content/drive/MyDrive/NLP/lstm_model6')

In [ ]:
y_pred = reconstructed_model.predict(X_test_pad)
y_pred2 = []
for i in y_pred:
  y_pred2.append(i[0])
import pickle
dbfile = open('lstm_predict2.pkl', 'wb')
  
# source, destination
pickle.dump(y_pred2, dbfile)                     
dbfile.close()

27/27 [==============================] - 16s 506ms/step
